In [1]:
import requests
from bs4 import BeautifulSoup
import re

Возьмем пробную категорию: Экономика

I. Функция, вычленяющая 'Окончание ссылки для итерации постоянной итерации'

In [2]:
def extract_link_pattern(inital_url):
    '''Функция возвращает ссылку, на следующую страницу (next) содержащую новости'''
    
    response = requests.get(inital_url).text

    html_structure = BeautifulSoup(response, 'html.parser')

    next_button = html_structure.findAll('a', class_= 'btn lenta_pager_next') #Содержит HTML-тэг, в формате строки

    pattern = r"\?down=\d+"

    try:
        return re.findall(pattern, str(next_button[0]))[0]
    except:
        return 0

II. Цикл, формирующий список, состоящий из всех страниц новостей по конкретной тематике

In [ ]:
initial_url = 'https://vtomske.ru/tag/economics' #Начальная ссылка

string_to_be_formatted = 'https://vtomske.ru/tag/economics{}'

links_list = []

links_list.append(initial_url)

#while len(links_list) <= 5000:
while True:

    next_pattern = extract_link_pattern(initial_url) #Переменная содержит в себе паттерн продолжения

    print(next_pattern)

    if next_pattern == 0:
        break #Цикл прекращается, если на странице отсутсвует кнопка 'Раньше' == Новости по тематике закончились.

    next_link = string_to_be_formatted.format(next_pattern) #Форматируем строку - ссылку, добавляя в нее продолжение

    links_list.append(next_link) #Сохраняем ссылку в список ссылок

    initial_url = next_link

In [190]:
def form_link_list(initial_url):
    '''Функция получает начальную ссылку на новостную тематику с сайта vtomske.ru в след. формате:
    https://vtomske.ru/tag/russia
    И возвращает список ссылок на все страницы новостей по данной новостной тематике.'''
    
    #string_to_be_formatted = 'https://vtomske.ru/tag/economics{}'
    string_to_be_formatted = f"{initial_url}{{}}"

    links_list = []

    links_list.append(initial_url)

#while len(links_list) <= 5000:
    while True:
        next_pattern = extract_link_pattern(initial_url) #Переменная содержит в себе паттерн продолжения

        if next_pattern == 0:
            break #Цикл прекращается, если на странице отсутсвует кнопка 'Раньше' == Новости по тематике закончились.

        next_link = string_to_be_formatted.format(next_pattern) #Форматируем строку - ссылку, добавляя в нее продолжение

        links_list.append(next_link) #Сохраняем ссылку в список ссылок

        initial_url = next_link

    return links_list

In [192]:
test_link = 'https://vtomske.ru/tag/economics'

test_list = form_link_list(test_link)

print(f"По тематике Экономика на сайте vtomske.ru представлено {len(test_list)} страниц новостей")

По тематике Экономика на сайте vtomske.ru представлено 116 страниц новостей


III. Попытка получать именно новости с ссылок

Опишем функцию для получения ссылок на конкретные новости со страницы

In [96]:
def extract_news_links(url):
    '''Функция, позволяющая извлечь список URL для всех новостей, находящихся на странице.
    Возвращает список (python list), состоящий из завершенных URL'''
    response = requests.get(url).text

    soup = BeautifulSoup(response, 'html.parser')

    raw_list = soup.findAll('a', class_="lenta_material") #Содержит в список, из которого следует извлечь ссылку на новость

    extract_pattern = r'href="(.*?)"' #паттерн регулярного выражения, позволяющий выделить окончание новости

    urls_final_list = []

    for tag in raw_list:
        
        tag = str(tag)

        result = re.findall(extract_pattern, tag)

        urls_final_list.append(result[0])

    return urls_final_list
    #return link_list

In [160]:
#Пример использования

extract_news_links(links_list[50])

['/news/143235-gazprom-neft-podpisala-s-tomskoi-oblastyu-dorojnuyu-kartu-do-2020g',
 '/news/143198-tomskii-zavod-poluchit-130-mln-iz-gosfonda-na-sozdanie-importozameshchayushchei-produkcii',
 '/news/143173-chislo-vydannyh-jitelyam-tomskoi-oblasti-kreditnyh-kart-vyroslo-na-24',
 '/news/143138-tomskaya-oblast-hochet-vnedrit-mehanizmy-gchp-v-selhozbiznes',
 '/news/143096-pribyl-ttu-tomska-za-2016g-sostavila-14-tys-rub',
 '/news/143029-eshche-odna-kompaniya-iz-tomska-popala-pod-sankcii-so-storony-ssha',
 '/news/142973-tomskaya-oblast-ispolzuet-opyt-niderlandov-dlya-razvitiya-umnogo-goroda',
 '/news/142765-vlasti-predostavyat-subsidii-tomskomu-biznesu-dlya-vyhoda-na-vneshnii-rynok',
 '/news/142866-gazprombank-nameren-vkladyvat-dengi-i-profresursy-v-razvitie-tomskoi-oblasti',
 '/news/142642-tomskie-deputaty-otmenili-lgoty-po-zemelnomu-nalogu-dlya-94-byudjetnyh-organizacii',
 '/news/142632-deputaty-tomska-vydelili-18-3-mln-na-vykup-kotelnoi-v-loskutovo',
 '/news/142485-pochti-500-tomichei-ust

IV. Отлично, извлекаем заголовок и текст новости из новости

In [152]:
def extract_title_and_text(news_url):
    '''Функция извлекает следующий аспекты из новостной статьи:
    1. Новостной заголовок.
    2. Текст новостной статьи.
    3. Дату публикации.
    4. Тэги.
    На вход получает ссылку на новостную статью.'''

    beautiful_html_structure = BeautifulSoup(requests.get(news_url).text, 'html.parser')

    headline = beautiful_html_structure.findAll('h1')[0]

    article_text = beautiful_html_structure.findAll('p')[0]

    #Извлечение даты

    publication_date = beautiful.findAll('time', class_="material-date")

    date_pattern = r'\d{4}-\d{2}-\d{2}'

    date = re.findall(date_pattern, str(publication_date))[0]

    #/Извлечение даты

    #Извлечение тегов

    #Выделение тэгов

    tags_raw = beautiful.findAll('div', class_="material-tags")

    tags_pattern = r">\w+<"

    tags = re.findall(tags_pattern, str(tags_raw))

    for index in range(0, len(tags)):
        tags[index] = tags[index].replace(">", "").replace("<", "")

    #/Излвечение тэгов

    return headline, article_text, date, tags


In [156]:
extract_title_and_text(one_news_url)

(<h1>Томский вуз приобрел нефтяной участок в Хакасии</h1>,
 <p>Технологический инкубатор томского политеха получил право разработки Алтайского участка с запасами углеводородного сырья в Хакасии. Томичи участвовали в аукционе и приобрели участок на 128-ом шаге, предложив 16 миллионов 700 тысяч рублей. Стартовая стоимость участка – 400 тысяч рублей. <br/>
 <br/>
 Это не первый углеводородный участок, приобретенный политехом в этом регионе. Как рассказали в управлении по недропользованию республики, томский инкубатор в мае приступит к геофизическим работам на участке Сохочульский площадью 100 кв. км. Алтайский участок находится в 25 км южнее Абакана. Его площадь составляет 261,25 кв. км. <br/>
 <br/>
 Разведка на этом участке проводилась в 60–е годы прошлого века. Результаты были получены неоднозначные. Поскольку использовались старые технологии, достоверно о запасах сказать нельзя. По предварительной оценке, здесь содержится 266 тыс. тонн нефти. Эксперты говорят, что это очень мало.<br/>